import

In [ ]:
import os
import numpy as np
from PIL import Image
import albumentations as A
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetLiteB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Configuration parameters
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 20
MODEL_PATH = 'olive_model.h5'
DATASET_PATH = 'augmented_images/'


ImportError: cannot import name 'EfficientNetLiteB0' from 'tensorflow.keras.applications' (/usr/local/lib/python3.11/dist-packages/keras/_tf_keras/keras/applications/__init__.py)

Image Import & Dataset Setup

In [ ]:
input_folder = "original_images/"
output_folder = "augmented_images/"
os.makedirs(output_folder, exist_ok=True)


NameError: name 'os' is not defined

augmentation

In [ ]:
# Albumentations pipeline for augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=20, p=0.5),
    A.GaussianBlur(p=0.2)
])

# Apply augmentation to images
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    image = np.array(Image.open(img_path).convert('RGB'))  # Ensure 3 channels
    for i in range(5):  # 5 augmentations per image
        augmented = transform(image=image)['image']
        Image.fromarray(augmented).save(f"{output_folder}/{img_name[:-4]}_aug{i}.jpg")

FileNotFoundError: [Errno 2] No such file or directory: 'original_images/'

Model Setup & Training

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import time
import json

class OliveClassifierEfficientNet:
    def __init__(self, img_height=224, img_width=224, batch_size=32):
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.model = None
        self.class_names = None
        self.history = None
        self.model_name = "EfficientNet-Lite0"

    def build_model(self, num_classes):
        """Build EfficientNet-Lite B0 model for olive classification"""
        # Import EfficientNet-Lite0 model
        try:
            # Using TF-Hub approach
            import tensorflow_hub as hub
            base_model = hub.KerasLayer(
                "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2",
                trainable=False
            )

            # Create new model with EfficientNet-Lite0 as base
            inputs = tf.keras.Input(shape=(self.img_height, self.img_width, 3))
            x = base_model(inputs)
            x = layers.Dropout(0.2)(x)
            x = layers.Dense(128, activation='relu')(x)
            x = layers.Dropout(0.2)(x)
            outputs = layers.Dense(num_classes, activation='softmax')(x)

            model = tf.keras.Model(inputs=inputs, outputs=outputs)

        except:
            # Fallback to EfficientNetB0 from Keras applications if TF-Hub import fails
            print("Falling back to EfficientNetB0 from Keras applications")
            from tensorflow.keras.applications import EfficientNetB0

            base_model = EfficientNetB0(
                weights='imagenet',
                include_top=False,
                input_shape=(self.img_height, self.img_width, 3)
            )

            # Freeze the base model layers
            base_model.trainable = False

            # Create new model
            model = models.Sequential([
                base_model,
                layers.GlobalAveragePooling2D(),
                layers.Dropout(0.2),
                layers.Dense(128, activation='relu'),
                layers.Dropout(0.2),
                layers.Dense(num_classes, activation='softmax')
            ])

            self.model_name = "EfficientNetB0"

        # Compile the model
        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def create_data_generators(self, data_dir):
        """Create data generators for training, validation, and test sets"""
        # Create training data generator with augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=0.2
        )

        # Create test data generator without augmentation
        test_datagen = ImageDataGenerator(rescale=1./255)

        # Create training and validation generators
        train_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )

        validation_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )

        # If test_dir is provided, create test generator
        test_dir = os.path.join(os.path.dirname(data_dir), 'test')
        if os.path.exists(test_dir):
            test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical',
                shuffle=False
            )
        else:
            test_generator = validation_generator

        # Save class names
        self.class_names = list(train_generator.class_indices.keys())

        return train_generator, validation_generator, test_generator

    def train(self, train_generator, validation_generator, epochs=15, callbacks=None):
        """Train the model"""
        if callbacks is None:
            # Define callbacks for model training
            callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.2,
                    patience=3,
                    min_lr=0.00001
                )
            ]

        # Start timing
        start_time = time.time()

        # Train the model
        history = self.model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=epochs,
            callbacks=callbacks
        )

        # Calculate training time
        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.2f} seconds")

        self.history = history
        return history

    def evaluate(self, test_generator):
        """Evaluate the model"""
        # Evaluate the model
        test_loss, test_accuracy = self.model.evaluate(test_generator)
        print(f"Test accuracy: {test_accuracy:.4f}")
        print(f"Test loss: {test_loss:.4f}")

        # Get predictions for confusion matrix
        steps = test_generator.samples // test_generator.batch_size + 1
        predictions = self.model.predict(test_generator, steps=steps)
        y_pred = np.argmax(predictions, axis=1)
        y_true = test_generator.classes[:len(y_pred)]  # Match length of predictions

        # Print classification report
        print("\nClassification Report:")
        report = classification_report(y_true, y_pred, target_names=self.class_names, output_dict=True)
        print(classification_report(y_true, y_pred, target_names=self.class_names))

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title(f'Confusion Matrix - {self.model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.show()

        return {
            'accuracy': test_accuracy,
            'loss': test_loss,
            'classification_report': report,
            'confusion_matrix': cm.tolist()
        }

    def plot_training_history(self):
        """Plot training and validation accuracy and loss"""
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        epochs_range = range(len(acc))

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.show()

    def save_model(self, filepath):
        """Save the model and metadata"""
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath) if os.path.dirname(filepath) else '.', exist_ok=True)

        # Save the model
        self.model.save(filepath)

        # Save class names and metadata
        metadata = {
            'class_names': self.class_names,
            'img_height': self.img_height,
            'img_width': self.img_width,
            'model_name': self.model_name,
            'date_created': time.strftime('%Y-%m-%d %H:%M:%S')
        }

        with open(f"{filepath}_metadata.json", "w") as f:
            json.dump(metadata, f)

        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        """Load a saved model and metadata"""
        # Load the model
        self.model = models.load_model(filepath, compile=True)

        # Load metadata
        try:
            with open(f"{filepath}_metadata.json", "r") as f:
                metadata = json.load(f)
                self.class_names = metadata['class_names']
                self.img_height = metadata['img_height']
                self.img_width = metadata['img_width']
                self.model_name = metadata.get('model_name', 'EfficientNet-Lite0')
        except FileNotFoundError:
            print("Warning: Metadata file not found. Class names may not be available.")

        print(f"Model loaded from {filepath}")

    def predict_image(self, image_path):
        """Predict the class of a single image"""
        # Check if image exists
        if not os.path.exists(image_path):
            print(f"Error: Image file '{image_path}' not found.")
            return None

        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(
            image_path, target_size=(self.img_height, self.img_width)
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) / 255.0

        # Make prediction
        predictions = self.model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])

        # Check if class names are available
        if self.class_names:
            predicted_class = self.class_names[predicted_class_index]
        else:
            predicted_class = f"Class {predicted_class_index}"

        confidence = float(predictions[0][predicted_class_index])

        # Display results
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Predicted: {predicted_class} ({confidence:.2%})")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Create result dictionary with all classes and probabilities
        results = {}
        if self.class_names:
            results = {self.class_names[i]: float(predictions[0][i]) for i in range(len(self.class_names))}
        else:
            results = {f"Class {i}": float(predictions[0][i]) for i in range(len(predictions[0]))}

        return {
            'predicted_class': predicted_class,
            'confidence': confidence,
            'all_probabilities': results,
            'image_path': image_path
        }

    def predict_batch(self, image_dir):
        """Predict classes for all images in a directory"""
        # Check if directory exists
        if not os.path.exists(image_dir):
            print(f"Error: Directory '{image_dir}' not found.")
            return None

        # Get all image files
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
        image_files = [
            os.path.join(image_dir, f) for f in os.listdir(image_dir)
            if os.path.splitext(f.lower())[1] in image_extensions
        ]

        if not image_files:
            print(f"No image files found in '{image_dir}'")
            return None

        # Predict each image
        results = []
        for image_path in image_files:
            print(f"Predicting {os.path.basename(image_path)}...")
            result = self.predict_image(image_path)
            if result:
                results.append(result)

        # Summarize results
        print(f"\nPredicted {len(results)} images")

        return results

# Example notebook usage
def run_notebook_example():
    # Replace these values with your own settings
    data_dir = "olive_images"  # Path to your training data
    model_path = "models/olive_classifier"  # Where to save the model
    epochs = 15
    batch_size = 32

    # Create classifier
    classifier = OliveClassifierEfficientNet(img_height=224, img_width=224, batch_size=batch_size)

    # Dataset creation and training
    if os.path.exists(model_path):
        print(f"Loading existing model from {model_path}...")
        classifier.load_model(model_path)
    elif os.path.exists(data_dir):
        print(f"Training new model with data from {data_dir}...")
        # Create data generators
        train_generator, validation_generator, test_generator = classifier.create_data_generators(data_dir)

        # Build model
        num_classes = len(train_generator.class_indices)
        print(f"Found {num_classes} olive classes: {list(train_generator.class_indices.keys())}")
        classifier.build_model(num_classes)

        # Print model summary
        classifier.model.summary()

        # Train model
        classifier.train(train_generator, validation_generator, epochs=epochs)

        # Plot training history
        classifier.plot_training_history()

        # Evaluate model
        eval_results = classifier.evaluate(test_generator)

        # Save model
        os.makedirs(os.path.dirname(model_path) if os.path.dirname(model_path) else '.', exist_ok=True)
        classifier.save_model(model_path)
    else:
        print(f"Error: Neither model path {model_path} nor data directory {data_dir} exists.")
        return

    # Return the classifier for further use
    return classifier

# This is what you'll run in your notebook
# classifier = run_notebook_example()

# To predict a single image:
# result = classifier.predict_image("path/to/your/olive_image.jpg")

# To predict multiple images in a directory:
# results = classifier.predict_batch("path/to/your/test_images")

# retrieve data from the ship

In [ ]:
# MicroPython Code (ESP32)
from umqtt.simple import MQTTClient
import ujson
from machine import UART

# OBD-II UART Setup
obd = UART(1, baudrate=9600, tx=17, rx=16)

# MQTT Setup
client = MQTTClient("car_123", "mqtt.broker.com")
client.connect()

while True:
    # Read OBD-II data (e.g., RPM)
    obd.write("010C\r\n")  # RPM PID
    response = obd.read()

    # Publish to MQTT
    payload = ujson.dumps({"rpm": response})
    client.publish("car/123/obd", payload)



ModuleNotFoundError: No module named 'umqtt'

ESP32-CAM (MicroPython code)

In [ ]:
import network
import socket
import time
from machine import Pin
import esp32
import camera

# Import configuration from separate file
try:
    import config
    SSID = config.WIFI_SSID
    PASSWORD = config.WIFI_PASSWORD
    PC_IP = config.SERVER_IP
    PC_PORT = config.SERVER_PORT
except ImportError:
    print("Config file not found. Please create config.py with your settings.")
    raise

# === Connect to Wi-Fi ===
def connect_wifi():
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect(SSID, PASSWORD)
    while not wlan.isconnected():
        print('Connecting to WiFi...')
        time.sleep(1)
    print('Connected:', wlan.ifconfig())

# === Camera Init ===
def init_camera():
    try:
        camera.init(0, format=camera.JPEG)
        camera.framesize(camera.FRAME_QVGA)  # Options: FRAME_QVGA, FRAME_VGA, etc.
        print("Camera initialized")
    except Exception as e:
        print("Camera init failed:", e)

# === Capture and Send ===
def capture_and_send():
    try:
        img = camera.capture()
        print(f"Captured image: {len(img)} bytes")

        s = socket.socket()
        s.connect((PC_IP, PC_PORT))
        s.send(img)
        s.close()
        print("Image sent")
    except Exception as e:
        print("Error sending image:", e)

# === Main ===
try:
    connect_wifi()
    init_camera()

    while True:
        capture_and_send()
        time.sleep(5)  # Send every 5 seconds
except KeyboardInterrupt:
    print("Program stopped by user")
    camera.deinit()
except Exception as e:
    print(f"Fatal error: {e}")
    camera.deinit()

ModuleNotFoundError: No module named 'network'

PC (Python script to receive image)

In [ ]:
# pc_receiver.py (Python on your PC)
import socket

HOST = '0.0.0.0'
PORT = 8080

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(1)

print(f"Listening on port {PORT}...")

img_counter = 0

while True:
    conn, addr = server.accept()
    print(f"Connection from {addr}")

    data = b''
    while True:
        chunk = conn.recv(4096)
        if not chunk:
            break
        data += chunk

    with open(f'image_{img_counter}.jpg', 'wb') as f:
        f.write(data)
        print(f'Saved image_{img_counter}.jpg')

    img_counter += 1
    conn.close()


OSError: [Errno 98] Address already in use

**Image Recognition**


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import time
import json

class OliveClassifierEfficientNet:
    def __init__(self, img_height=224, img_width=224, batch_size=32):
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.model = None
        self.class_names = None
        self.history = None
        self.model_name = "EfficientNet-Lite0"

    def build_model(self, num_classes):
        """Build EfficientNet-Lite B0 model for olive classification"""
        # Import EfficientNet-Lite0 model
        try:
            # Using TF-Hub approach
            import tensorflow_hub as hub
            base_model = hub.KerasLayer(
                "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2",
                trainable=False
            )

            # Create new model with EfficientNet-Lite0 as base
            inputs = tf.keras.Input(shape=(self.img_height, self.img_width, 3))
            x = base_model(inputs)
            x = layers.Dropout(0.2)(x)
            x = layers.Dense(128, activation='relu')(x)
            x = layers.Dropout(0.2)(x)
            outputs = layers.Dense(num_classes, activation='softmax')(x)

            model = tf.keras.Model(inputs=inputs, outputs=outputs)

        except:
            # Fallback to EfficientNetB0 from Keras applications if TF-Hub import fails
            print("Falling back to EfficientNetB0 from Keras applications")
            from tensorflow.keras.applications import EfficientNetB0

            base_model = EfficientNetB0(
                weights='imagenet',
                include_top=False,
                input_shape=(self.img_height, self.img_width, 3)
            )

            # Freeze the base model layers
            base_model.trainable = False

            # Create new model
            model = models.Sequential([
                base_model,
                layers.GlobalAveragePooling2D(),
                layers.Dropout(0.2),
                layers.Dense(128, activation='relu'),
                layers.Dropout(0.2),
                layers.Dense(num_classes, activation='softmax')
            ])

            self.model_name = "EfficientNetB0"

        # Compile the model
        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def create_data_generators(self, data_dir):
        """Create data generators for training, validation, and test sets"""
        # Create training data generator with augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=0.2
        )

        # Create test data generator without augmentation
        test_datagen = ImageDataGenerator(rescale=1./255)

        # Create training and validation generators
        train_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )

        validation_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )

        # If test_dir is provided, create test generator
        test_dir = os.path.join(os.path.dirname(data_dir), 'test')
        if os.path.exists(test_dir):
            test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical',
                shuffle=False
            )
        else:
            test_generator = validation_generator

        # Save class names
        self.class_names = list(train_generator.class_indices.keys())

        return train_generator, validation_generator, test_generator

    def train(self, train_generator, validation_generator, epochs=15, callbacks=None):
        """Train the model"""
        if callbacks is None:
            # Define callbacks for model training
            callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.2,
                    patience=3,
                    min_lr=0.00001
                )
            ]

        # Start timing
        start_time = time.time()

        # Train the model
        history = self.model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=epochs,
            callbacks=callbacks
        )

        # Calculate training time
        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.2f} seconds")

        self.history = history
        return history

    def evaluate(self, test_generator):
        """Evaluate the model"""
        # Evaluate the model
        test_loss, test_accuracy = self.model.evaluate(test_generator)
        print(f"Test accuracy: {test_accuracy:.4f}")
        print(f"Test loss: {test_loss:.4f}")

        # Get predictions for confusion matrix
        steps = test_generator.samples // test_generator.batch_size + 1
        predictions = self.model.predict(test_generator, steps=steps)
        y_pred = np.argmax(predictions, axis=1)
        y_true = test_generator.classes[:len(y_pred)]  # Match length of predictions

        # Print classification report
        print("\nClassification Report:")
        report = classification_report(y_true, y_pred, target_names=self.class_names, output_dict=True)
        print(classification_report(y_true, y_pred, target_names=self.class_names))

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title(f'Confusion Matrix - {self.model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.show()

        return {
            'accuracy': test_accuracy,
            'loss': test_loss,
            'classification_report': report,
            'confusion_matrix': cm.tolist()
        }

    def plot_training_history(self):
        """Plot training and validation accuracy and loss"""
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        epochs_range = range(len(acc))

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.show()

    def save_model(self, filepath):
        """Save the model and metadata"""
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath) if os.path.dirname(filepath) else '.', exist_ok=True)

        # Save the model
        self.model.save(filepath)

        # Save class names and metadata
        metadata = {
            'class_names': self.class_names,
            'img_height': self.img_height,
            'img_width': self.img_width,
            'model_name': self.model_name,
            'date_created': time.strftime('%Y-%m-%d %H:%M:%S')
        }

        with open(f"{filepath}_metadata.json", "w") as f:
            json.dump(metadata, f)

        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        """Load a saved model and metadata"""
        # Load the model
        self.model = models.load_model(filepath, compile=True)

        # Load metadata
        try:
            with open(f"{filepath}_metadata.json", "r") as f:
                metadata = json.load(f)
                self.class_names = metadata['class_names']
                self.img_height = metadata['img_height']
                self.img_width = metadata['img_width']
                self.model_name = metadata.get('model_name', 'EfficientNet-Lite0')
        except FileNotFoundError:
            print("Warning: Metadata file not found. Class names may not be available.")

        print(f"Model loaded from {filepath}")

    def predict_image(self, image_path):
        """Predict the class of a single image"""
        # Check if image exists
        if not os.path.exists(image_path):
            print(f"Error: Image file '{image_path}' not found.")
            return None

        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(
            image_path, target_size=(self.img_height, self.img_width)
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) / 255.0

        # Make prediction
        predictions = self.model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])

        # Check if class names are available
        if self.class_names:
            predicted_class = self.class_names[predicted_class_index]
        else:
            predicted_class = f"Class {predicted_class_index}"

        confidence = float(predictions[0][predicted_class_index])

        # Display results
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Predicted: {predicted_class} ({confidence:.2%})")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Create result dictionary with all classes and probabilities
        results = {}
        if self.class_names:
            results = {self.class_names[i]: float(predictions[0][i]) for i in range(len(self.class_names))}
        else:
            results = {f"Class {i}": float(predictions[0][i]) for i in range(len(predictions[0]))}

        return {
            'predicted_class': predicted_class,
            'confidence': confidence,
            'all_probabilities': results,
            'image_path': image_path
        }

    def predict_batch(self, image_dir):
        """Predict classes for all images in a directory"""
        # Check if directory exists
        if not os.path.exists(image_dir):
            print(f"Error: Directory '{image_dir}' not found.")
            return None

        # Get all image files
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
        image_files = [
            os.path.join(image_dir, f) for f in os.listdir(image_dir)
            if os.path.splitext(f.lower())[1] in image_extensions
        ]

        if not image_files:
            print(f"No image files found in '{image_dir}'")
            return None

        # Predict each image
        results = []
        for image_path in image_files:
            print(f"Predicting {os.path.basename(image_path)}...")
            result = self.predict_image(image_path)
            if result:
                results.append(result)

        # Summarize results
        print(f"\nPredicted {len(results)} images")

        return results